In [56]:
% matplotlib inline

In [57]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
import time

In [58]:
advertiserlist_path = '../data/adtech_intern_advertiserlist.csv'
log_path = '../data/intern_samplelog.csv'

In [59]:
advertiserlist = pd.read_csv(advertiserlist_path)
log_original = pd.read_csv(log_path)
log = log_original

In [60]:
log.loc[:,'os'][log.os=='iOS'] = 0
log.loc[:,'os'][log.os=='Android'] = 1

/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [61]:
def hashnum(modnum, data):
    mod = 10 ** modnum
    return hash(data) % mod

In [62]:
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1).fillna(value=0) for ad in xrange(1, 11)]
input_type='string'
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
modnum = 3
n_folds = 10

for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    print xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)
    xgb_model = xgb.train(param, dlog, num_round)
    model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(xgb_model, model_path)

----------------- Advertiser 1 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.650861      0.006171        0.654020       0.001627
1       0.666785      0.003373        0.669649       0.002339
----------------- Advertiser 2 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.554345      0.006989        0.558015       0.006110
1       0.567882      0.004680        0.573683       0.002197
----------------- Advertiser 3 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.616817      0.004229        0.618254       0.001042
1       0.636157      0.003911        0.639709       0.000679
----------------- Advertiser 4 -------------------
   test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0       0.561811      0.011675        0.565452       0.010532
1       0.584164      0.005161        0.590309       0.003335
----------------- Advertiser 5 -------------------
 

In [63]:
xgb_models = [joblib.load('../models/Ad%d_xgb.pkl'%ind) for ind in xrange(1,11)]

In [51]:
test_samples = log.drop(['click', 'advertiser'], axis=1).loc[:10000].fillna(value=0)
start_time = time.time()

for test_sample in test_samples.as_matrix():
    if test_sample[3] == 'iOS':
        test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 0])
    else:
        test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 1])
    [float(model.predict(test_sample))for model in xgb_models]
print time.time() - start_time

25.8333959579


In [80]:
sample_dict = {
    'id' : "ididid",
    'floorPrice' : 1000,
    'site' : "medianame",
    'device' : "device",
    'user' : "a9102910201",
    'test' : 0
}

In [84]:
if sample_dict['floorPrice'] == 'nan':
    sample_dict['floorPrice'] = 0
if sample_dict['device'] == 'iOS':
        test_sample = xgb.DMatrix([hashnum(modnum, sample_dict['site']), hashnum(modnum, sample_dict['user']), 0,  sample_dict['floorPrice']])
else:
    test_sample = xgb.DMatrix([hashnum(modnum, sample_dict['site']), hashnum(modnum, sample_dict['user']), 1,  sample_dict['floorPrice']])
[float(model.predict(test_sample))for model in xgb_models]

[0.21913763880729675,
 0.14578142762184143,
 0.18474996089935303,
 0.19934862852096558,
 0.7245240211486816,
 0.24230889976024628,
 0.6336053609848022,
 0.22833609580993652,
 0.19325865805149078,
 0.8674460053443909]